#### Comparative Similarity Estimation
What are we Doing? This is essentially a fact finding experiment.

We want to estimate the possibility of using the current corpus as part of a model to evaluate potential future articles
<div>
   <img src="../../../images/BERTScoring3.png" height="374"; width="374">
</div>

In [5]:
import sys
import os
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"
module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 

sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation/BERTScore', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/home/deleidos/Notebooks/lib']


### Preparation

Note, we are now using the practice of centralising python libraries specific to this set of Jupyter Notebooks in one location. This is possible because Python uses the concept of module naming and location as shown in the graphic below
<div>
   <img src="../../../images/SysPath.jpg" height="374"; width="374">
</div>

In [6]:
#!pip install sentence_transformers
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import time
import datetime
from datetime import datetime as DT
import json
import random
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import nltk
# nltk.download('punkt')
# nltk.download('brown')
# Locals 
from pathlib import Path 
import os, sys 
import glob

from IPython.display import Image
from IPython.core.display import HTML 
from sentence_transformers import SentenceTransformer, util

# nltk.download('punkt')
###
module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path)  
print (f"sys.path : {sys.path}") 

import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT
from os import listdir
import os.path as osp
from os.path import isfile, join

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import pandas as pd
import numpy as np
# regex
import re
p = re.compile('[a-z]+')
p
re.compile('[a-z]+')
## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"

from APIutils import *
from dynamoDButils import *
# from utils import *

###
def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")

complete(state='imports done')


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation/BERTScore', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/home/deleidos/Notebooks/lib']

Cell imports done @ 2023-02-03 19:54:41


[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [12]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnf='semanticSimArticlesTestingME.json' 


In [13]:
jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
# xposedf = df.transpose() # pivot the matrix

In [15]:
df.head(10)

,source,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg
0,mixed_summary1,race governor mansion state today gop could en...,0.542,0.455,0.445,0.440,0.432,0.44300,0.367909,0.353988,0.320898,0.317035,0.306390,0.333244
1,mixed_summary2,turn facebook guilty half accused gay kiss inc...,0.482,0.449,0.440,0.430,0.427,0.43650,0.446461,0.445366,0.437336,0.412640,0.407001,0.429761
2,mixed_summary3,big fan southern california neither northern c...,0.419,0.400,0.374,0.365,0.362,0.38400,0.337420,0.306980,0.305775,0.298095,0.286152,0.299250
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.647,0.629,0.628,0.616,0.606,0.62520,0.564485,0.556181,0.552514,0.547629,0.544024,0.552966
4,mixed_summary5,supreme court facing docket high profile polit...,0.432,0.413,0.380,0.378,0.372,0.39500,0.357034,0.350235,0.347016,0.343617,0.340958,0.347772
5,mixed_summary6,british soldier left new york city george wash...,0.376,0.351,0.316,0.298,0.286,0.32540,0.342358,0.332993,0.325036,0.298862,0.291214,0.318093
6,mixed_summary7,israel launched round airstrikes gaza today wo...,0.681,0.653,0.653,0.635,0.624,0.64125,0.574472,0.550546,0.548333,0.544247,0.535290,0.544604
7,mixed_summary8,picasso painting found vanished paris museum d...,0.339,0.331,0.319,0.316,0.315,0.32400,0.352555,0.349328,0.348217,0.347080,0.333333,0.349295
8,mixed_summary9,dispute freshness wendy fry leaf year old minn...,0.435,0.418,0.409,0.404,0.397,0.41260,0.419426,0.411391,0.411290,0.391297,0.384195,0.403520
9,mixed_summary10,year old dallas police officer died wednesday ...,0.515,0.486,0.465,0.446,0.446,0.47160,0.447424,0.440548,0.408651,0.407450,0.401035,0.421022


In [17]:
def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min())) # *10
#     print (f"dataset-dataset.min() : {dataset-dataset.min()}")
#     print (f"(dataset.max()-dataset.min()): {(dataset.max()-dataset.min())}")
    print (f"dataset{dataset}", end="\n\n")
    print (f"dataNorm: {dataNorm}", end="\n\n")
#     dataNorm["article"]=dataset["article"]
#     dataNorm["summary"]=dataset["summary"]
    return dataNorm
def computeMean (df):
    AXIS = 1
    df['hfAvg'] = df[['hf1', 'hf2']].mean(axis=AXIS)
    df['kbAvg'] = df[['kb1', 'kb2']].mean(axis=AXIS)
#     df['topicAvg'] = df[['score1', 'score2']].mean(axis=AXIS)
#     print (f"df : {df}")
    return df
def displaySelectedCols(df, cols):
    return df[ cols ].copy()

def normalizeScaled(df_min_max_scaled, column): # Note must do per column!
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
    return df_min_max_scaled

In [23]:
df2 = df[
    [
         'hf1', 'hf2', 'hf3', 'hf4', 'hf5', 'hfAvg'
        ,'kb1', 'kb2', 'kb3', 'kb4', 'kb5', 'kbAvg']].copy()
#         , 'score1','score2']
#         ].copy()
# df2['topicAvg'] = None
# print (f"df 2 : {df2.head(20)}")
# compute mean 
df2 = computeMean(df2)
# print (f"df 3 post mean: {df3.head(20)}")
df2 = normalize(df2)
df2 = df2.copy()

df2['source'] = df['source']
df2['summary'] = df['summary']
# df2['topic1'] = df['topic1']
# df2['topic2'] = df['topic2']

cols =  ['source', 'summary'
       ,'hf1', 'hf2', 'hfAvg'
       ,'kb1', 'kb2', 'kbAvg'
#        ,'topic1','score1','topic2','score2', 'topicAvg'
    ]
df2= displaySelectedCols(df2, cols)

pd.set_option('display.float_format','{:.2f}'.format)
df2.head(10)
# display (df2)

dataset      hf1  hf2  hf3  hf4  hf5  hfAvg  kb1  kb2  kb3  kb4  kb5  kbAvg
0    0.54 0.46 0.44 0.44 0.43   0.50 0.37 0.35 0.32 0.32 0.31   0.36
1    0.48 0.45 0.44 0.43 0.43   0.47 0.45 0.45 0.44 0.41 0.41   0.45
2    0.42 0.40 0.37 0.37 0.36   0.41 0.34 0.31 0.31 0.30 0.29   0.32
3    0.65 0.63 0.63 0.62 0.61   0.64 0.56 0.56 0.55 0.55 0.54   0.56
4    0.43 0.41 0.38 0.38 0.37   0.42 0.36 0.35 0.35 0.34 0.34   0.35
...   ...  ...  ...  ...  ...    ...  ...  ...  ...  ...  ...    ...
5617 0.53 0.45 0.45 0.42 0.42   0.49 0.42 0.40 0.39 0.38 0.38   0.41
5618 0.30 0.29 0.26 0.25 0.25   0.30 0.28 0.26 0.26 0.25 0.25   0.27
5619 0.36 0.34 0.33 0.32 0.28   0.35 0.35 0.35 0.34 0.34 0.34   0.35
5620 0.53 0.47 0.47 0.46 0.46   0.50 0.45 0.45 0.44 0.44 0.44   0.45
5621 0.45 0.45 0.44 0.44 0.44   0.45 0.44 0.44 0.42 0.41 0.41   0.44

[5622 rows x 12 columns]

dataNorm:       hf1  hf2  hf3  hf4  hf5  hfAvg  kb1  kb2  kb3  kb4  kb5  kbAvg
0    0.52 0.42 0.46 0.47 0.49   0.48 0.34 0.33 0.31 0.32 0.

,source,summary,hf1,hf2,hfAvg,kb1,kb2,kbAvg
0,mixed_summary1,race governor mansion state today gop could en...,0.52,0.42,0.48,0.34,0.33,0.33
1,mixed_summary2,turn facebook guilty half accused gay kiss inc...,0.43,0.41,0.42,0.48,0.50,0.49
2,mixed_summary3,big fan southern california neither northern c...,0.34,0.34,0.34,0.28,0.24,0.26
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.68,0.70,0.69,0.70,0.70,0.70
4,mixed_summary5,supreme court facing docket high profile polit...,0.36,0.36,0.36,0.32,0.32,0.32
5,mixed_summary6,british soldier left new york city george wash...,0.27,0.26,0.27,0.29,0.29,0.29
6,mixed_summary7,israel launched round airstrikes gaza today wo...,0.74,0.73,0.74,0.72,0.69,0.71
7,mixed_summary8,picasso painting found vanished paris museum d...,0.22,0.23,0.22,0.31,0.32,0.32
8,mixed_summary9,dispute freshness wendy fry leaf year old minn...,0.36,0.37,0.36,0.44,0.43,0.44
9,mixed_summary10,year old dallas police officer died wednesday ...,0.48,0.47,0.48,0.49,0.49,0.49


In [26]:
# Now take a look at using min-max feature scaling
# The min-max approach (often called normalization) rescales the feature 
# to a hard and fast range of [0,1] by subtracting the minimum value of the feature 
# then dividing by the range. We can apply the min-max scaling in Pandas using the .min() and .max() methods.
columns = [
         'hf1', 'hf2', 'hf3', 'hf4', 'hf5', 'hfAvg'
        ,'kb1', 'kb2', 'kb3', 'kb4', 'kb5', 'kbAvg']
#         , 'score1','score2']
df_min_max_scaled = df[ columns ].copy()
# df_min_max_scaled['topicAvg'] = None
# display(df)
# apply normalization techniques by Column 1
column = 'hf1'
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'hf2' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'kb1' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'kb2' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)

df_min_max_scaled = computeMean(df_min_max_scaled)

# view normalized data
# display(df_min_max_scaled)
# ###
df_min_max_scaled['source'] = df['source']
df_min_max_scaled['summary'] = df['summary']
# df_min_max_scaled['topic1'] = df['topic1']
# df_min_max_scaled['topic2'] = df['topic2']

cols =  [ 'source', 'summary'
       ,'hf1', 'hf2', 'hfAvg'
       ,'kb1', 'kb2', 'kbAvg'
#        ,'topic1','score1','topic2','score2', 'topicAvg'
    ]
df_min_max_scaled= displaySelectedCols(df_min_max_scaled, cols)

pd.set_option('display.float_format','{:.2f}'.format)
df_min_max_scaled.head(20)
# display (df2)


,source,summary,hf1,hf2,hfAvg,kb1,kb2,kbAvg
0,mixed_summary1,race governor mansion state today gop could en...,0.52,0.42,0.47,0.34,0.33,0.33
1,mixed_summary2,turn facebook guilty half accused gay kiss inc...,0.43,0.41,0.42,0.48,0.50,0.49
2,mixed_summary3,big fan southern california neither northern c...,0.34,0.34,0.34,0.28,0.24,0.26
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.68,0.70,0.69,0.70,0.70,0.70
4,mixed_summary5,supreme court facing docket high profile polit...,0.36,0.36,0.36,0.32,0.32,0.32
5,mixed_summary6,british soldier left new york city george wash...,0.27,0.26,0.27,0.29,0.29,0.29
6,mixed_summary7,israel launched round airstrikes gaza today wo...,0.74,0.73,0.73,0.72,0.69,0.70
7,mixed_summary8,picasso painting found vanished paris museum d...,0.22,0.23,0.22,0.31,0.32,0.32
8,mixed_summary9,dispute freshness wendy fry leaf year old minn...,0.36,0.37,0.36,0.44,0.43,0.43
9,mixed_summary10,year old dallas police officer died wednesday ...,0.48,0.47,0.48,0.49,0.49,0.49


In [28]:
df_min_max_scaled

,source,summary,hf1,hf2,hfAvg,kb1,kb2,kbAvg
0,mixed_summary1,race governor mansion state today gop could en...,0.52,0.42,0.47,0.34,0.33,0.33
1,mixed_summary2,turn facebook guilty half accused gay kiss inc...,0.43,0.41,0.42,0.48,0.50,0.49
2,mixed_summary3,big fan southern california neither northern c...,0.34,0.34,0.34,0.28,0.24,0.26
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.68,0.70,0.69,0.70,0.70,0.70
4,mixed_summary5,supreme court facing docket high profile polit...,0.36,0.36,0.36,0.32,0.32,0.32
...,...,...,...,...,...,...,...,...
5617,mixed_summary5618,traditional end summit group photo year gather...,0.51,0.41,0.46,0.43,0.42,0.42
5618,mixed_summary5619,sofia coppola scored historic victory cannes f...,0.16,0.17,0.16,0.18,0.16,0.17
5619,mixed_summary5620,duck boat sinking killed missouri lake subject...,0.25,0.24,0.25,0.31,0.32,0.31
5620,mixed_summary5621,note tweeting politician watch post politwoops...,0.50,0.44,0.47,0.50,0.50,0.50


In [21]:
#Candidates in common:: Which techniques commonly select the same articles?
# TECHLABEL='technology'
# INNOLABEL='innovation'
# BUSILABEL='business innovation'
# SCIXLABEL= 'science'
dfClassifier = df_min_max_scaled.copy()
newdf = dfClassifier.loc[
    (
#         (
#         ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==INNOLABEL))  |
#         ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==BUSILABEL))  |
#         ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==SCIXLABEL))  |
#         ((dfClassifier['topic1']==BUSILABEL) & (dfClassifier['topic2']==TECHLABEL))  |
#         ((dfClassifier['topic1']==SCIXLABEL) & (dfClassifier['topic2']==TECHLABEL))  |        
#         ((dfClassifier['topic1']==INNOLABEL) & (dfClassifier['topic2']==TECHLABEL)) 
#             &
#         (dfClassifier['topicAvg'] >= 0.3) 
#             &
        (dfClassifier['hfAvg'] >= 0.5)
            & 
        (dfClassifier['kbAvg'] >= 0.4)
        
        ) , ['source', 'summary', 'hfAvg', 'kbAvg']
#     )
#         , ['article','hfAvg','kbAvg','topic1','topic2', 'topicAvg']
    ]

print (f"Common Selections : {len(newdf)}")
newdf


Common Selections : 788


,source,summary,hfAvg,kbAvg
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.69,0.70
6,mixed_summary7,israel launched round airstrikes gaza today wo...,0.73,0.70
13,mixed_summary14,wall street journal today take look life death...,0.53,0.46
61,mixed_summary62,credit card purchase get little slower lot sec...,0.65,0.62
70,mixed_summary71,rookie cop nypd face criminal charge shooting ...,0.62,0.57
...,...,...,...,...
5601,mixed_summary5602,janet yellen getting lot grief first news conf...,0.62,0.62
5602,mixed_summary5603,tip nypd put cellphone pillow sleeping chargin...,0.54,0.45
5604,mixed_summary5605,mers cov potentially deadly virus spread middl...,0.65,0.70
5609,mixed_summary5610,microsoft unleashed window world today form co...,0.56,0.54


In [25]:
# Calculate percentage of candidates that were included
percent_select = (len(newdf)/len(df))*100

print (f"Percent Selected : {round(percent_select, 2)}%")

Percent Selected : 14.02%
